# Voyages API Example

## Run this example in [Colab](https://colab.research.google.com/github/SignalOceanSdk/SignalSDK/blob/master/docs/examples/jupyter/VoyagesAPI/VoyagesAPI.ipynb). 

## Setup
Install the Signal Ocean SDK:
```
pip install signal-ocean
```
Set your subscription key acquired here: https://apis.signalocean.com/profile

In [1]:
!pip install signal-ocean

In [2]:
signal_ocean_api_key = '6de2b1078a424f5cb70f571948fc4ba1' #replace with your subscription key

## Call the Voyages API
The Voyages API retrieves information about vessel voyages.

In [3]:
from signal_ocean import Connection
from signal_ocean.voyages import VoyagesAPI
from signal_ocean.voyages import Vessel, VesselFilter
from signal_ocean.voyages import VesselType, VesselTypeFilter
from signal_ocean.voyages import VesselClass, VesselClassFilter
import pandas as pd
from datetime import date, timedelta

In [4]:
connection = Connection(signal_ocean_api_key)
api = VoyagesAPI(connection)

### Get voyages for vessel

In [5]:
start_date_from = None
endpoint = api._get_endpoint(imo=[91017,90455],
        voyage_keys=[2,3],
        event_type=2,
        event_horizons=[0,1,2],
        event_purpose=["purpose","purpose1","purpose2"],
        vessel_class_id=[84,60,61,84],
        port_ids=[2,3,4],
        vessel_type_id=3,
        start_date_from='2021-06-04',
        start_date_to='2021-07-04',
        first_load_arrival_date_from='2021-06-04',
        first_load_arrival_date_to='2021-07-04',
        end_date_from='2021-06-04',
        end_date_to='2021-07-04',
        market_info_rate_type="type",
        commercial_operator_id=2,
        charterer_id=2,
        voyage_horizon=[0,2,1],
        token="token",
        hide_event_details=True,
        hide_events=True,
        hide_market_info=False)
print(endpoint)

voyages-api/v3/voyages/nested?Imo=91017&Imo=90455&VoyageKeys=2&VoyageKeys=3&EventType=2&EventHorizons=0&EventHorizons=1&EventHorizons=2&EventPurpose=purpose&EventPurpose=purpose1&EventPurpose=purpose2&VesselClassId=84&VesselClassId=60&VesselClassId=61&VesselClassId=84&PortIds=2&PortIds=3&PortIds=4&VesselTypeId=3&StartDateFrom=2021-06-04&StartDateTo=2021-07-04&FirstLoadArrivalDateFrom=2021-06-04&FirstLoadArrivalDateTo=2021-07-04&EndDateFrom=2021-06-04&EndDateTo=2021-07-04&MarketInfoRateType=type&CommercialOperatorId=2&ChartererId=2&VoyageHorizon=0&VoyageHorizon=2&VoyageHorizon=1&Token=token&HideEventDetails=True&HideEvents=True&HideMarketInfo=False


In [6]:
# get the imo by using the vessel name
vessel = api.get_imos(VesselFilter('Seaking'))[0]
vessel

Vessel(imo=9292187, vessel_name='Seaking')

In [7]:
vessel_voyages = api.get_voyages(imo=vessel.imo)
vessel_voyages

(Voyage(imo=9292187, voyage_number=1, vessel_type_id=1, vessel_class_id=84, vessel_status_id=1, commercial_operator_id=1684, deleted=False, events=(VoyageEvent(id='I8DC99BSEC1198000', port_id=3255, voyage_id=None, event_type='VoyageStart', event_horizon='Historical', purpose='Start', event_date=datetime.datetime(2009, 2, 10, 17, 21, 24, tzinfo=datetime.timezone.utc), arrival_date=None, sailing_date=None, latitude=27.8709, longitude=-78.897, geo_asset_id=5281, geo_asset_name='Freeport Shipyard', port_name='Freeport', country_id=39, country='Bahamas', area_idlevel0=24746, area_name_level0='Caribs', area_idlevel1=9, area_name_level1='Caribs', area_idlevel2=25019, area_name_level2='Atlantic America', area_idlevel3=25028, area_name_level3='West', low_ais_density=None, event_details=None), VoyageEvent(id='I8DC99BTEC1198000', port_id=3255, voyage_id=None, event_type='Stop', event_horizon='Historical', purpose='Stop', event_date=None, arrival_date=datetime.datetime(2009, 2, 12, 7, 54, 48, tzin

In [8]:
vessel_voyages_df = pd.DataFrame([v.__dict__ for v in vessel_voyages])
vessel_voyages_df.tail(5)

,imo,voyage_number,vessel_type_id,vessel_class_id,vessel_status_id,commercial_operator_id,deleted,events,id,horizon_id,...,is_implied_by_ais,has_manual_entries,ballast_distance,predicted_ballast_distance,laden_distance,predicted_laden_distance,suez_crossing,panama_crossing,canakkale_crossing,bosporus_crossing
61,9292187,62,1,84,1,1684,False,"(VoyageEvent(id='I8DC99BSEDA6FD400', port_id=7...",I8DC99BVEDA6FD400,1,...,None,None,4523.36,NaN,4586.67,NaN,None,None,None,None
62,9292187,63,1,84,1,1684,False,"(VoyageEvent(id='I8DC99BSEDA976100', port_id=3...",I8DC99BVEDA976100,1,...,None,None,4531.50,NaN,3968.99,NaN,None,None,None,None
63,9292187,64,1,84,1,1684,False,"(VoyageEvent(id='I8DC99BSEDABEEE00', port_id=3...",I8DC99BVEDABEEE00,1,...,None,None,3831.88,NaN,4588.66,NaN,None,None,None,None
64,9292187,65,1,84,1,1684,False,"(VoyageEvent(id='I8DC99BSEDB0E0800', port_id=7...",I8DC99BVEDB0E0800,1,...,None,None,4612.85,NaN,5273.31,NaN,None,None,None,None
65,9292187,66,1,84,1,1684,False,"(VoyageEvent(id='I8DC99BSEDB359500', port_id=3...",I8DC99BVEDB359500,2,...,None,None,4681.09,678.62,NaN,4443.77,None,None,None,None


In [8]:
vessel_events_df = pd.DataFrame(e.__dict__ for voyage_events in vessel_voyages_df['events'] for e in voyage_events)
vessel_events_df.tail(5)

,id,port_id,voyage_id,event_type,event_horizon,purpose,event_date,arrival_date,sailing_date,latitude,...,area_idlevel0,area_name_level0,area_idlevel1,area_name_level1,area_idlevel2,area_name_level2,area_idlevel3,area_name_level3,low_ais_density,event_details
382,I8DC99BTEDB1B3700,3711,None,Stop,Historical,Stop,NaT,2022-12-08 15:57:28+00:00,2022-12-09 11:52:36+00:00,23.8024,...,24777,Arabian Gulf,2,Arabian Gulf,89,Arabian Gulf,84,East,None,"(VoyageEventDetail(id='I8DC99BLEDB1B3700', eve..."
383,I8DC99BTEDB21CE80,3778,None,PortCall,Historical,Load,NaT,2022-12-11 23:46:36+00:00,2022-12-13 11:50:08+00:00,26.9431,...,24777,Arabian Gulf,2,Arabian Gulf,89,Arabian Gulf,84,East,None,"(VoyageEventDetail(id='I8DC99BLEDB21CE80', eve..."
384,I8DC99BTEDB359500,3295,None,PortCall,Historical,Discharge,NaT,2023-01-01 03:57:12+00:00,2023-01-03 07:48:06+00:00,22.6770,...,24725,South China,17,China / Taiwan,99,Far East,84,East,None,"(VoyageEventDetail(id='I8DC99BLEDB359500', eve..."
385,I8DC99BSEDB359500,3295,None,VoyageStart,Historical,Start,2023-01-03 07:48:06+00:00,NaT,NaT,22.6770,...,24725,South China,17,China / Taiwan,99,Far East,84,East,None,None
386,I8DC99BTEDB4FF300,3153,None,Stop,Future,Stop,NaT,2023-01-20 23:00:00+00:00,2023-01-20 23:00:00+00:00,25.1975,...,24777,Arabian Gulf,2,Arabian Gulf,89,Arabian Gulf,84,East,None,()


In [9]:
vessel_event_details_df = pd.DataFrame(e.__dict__ for event_details in vessel_events_df['event_details'] for e in event_details or [])
vessel_event_details_df.tail(5)

,id,event_id,event_detail_type,arrival_date,sailing_date,start_time_of_operation,end_time_of_operation,sts_id,geo_asset_id,geo_asset_name,latitude,longitude,other_vessel_imo,other_vessel_name
349,I8DC99BLEDB00D900,None,Jetty,2022-11-20 03:48:51+00:00,2022-11-23 11:51:53+00:00,2022-11-21 03:58:04+00:00,2022-11-23 11:51:53+00:00,None,7243,Rayong Ldpe Plant,12.6359,101.3018,NaN,None
350,I8DC99BLEDB0E0800,None,Stop,2022-11-26 03:59:53+00:00,2022-11-27 03:57:36+00:00,NaT,NaT,None,4446,Changi Lightering Zone,1.2670,103.9481,NaN,None
351,I8DC99BLEDB1B3700,None,Stop,2022-12-08 15:57:28+00:00,2022-12-09 11:52:36+00:00,NaT,NaT,None,5101,Mina Al Fahal Refinery,23.8024,59.4541,NaN,None
352,I8DC99BLEDB21CE80,None,Jetty,2022-12-11 23:46:36+00:00,2022-12-13 11:50:08+00:00,2022-12-11 23:46:36+00:00,2022-12-13 11:50:08+00:00,None,5868,Saudi Aramco Ju'aymah Crude Oil Terminal,26.9431,50.0748,NaN,None
353,I8DC99BLEDB359500,None,Jetty,2023-01-01 03:57:12+00:00,2023-01-03 07:48:06+00:00,2023-01-01 07:53:49+00:00,2023-01-03 07:48:06+00:00,None,5191,Huizhou Refinery,22.6770,114.6516,NaN,None


### Get voyages for vessel class

In [30]:
#get vessel class id for vlcc
vc = api.get_vessel_classes(VesselClassFilter('suezmax'))[0]
vlcc_id = vc.vessel_class_id
vlcc_id

85

In [34]:
date_from = date.today() - timedelta(days=180)
recent_vlcc_voyages = api.get_voyages(vessel_class_id=vlcc_id, date_from=date_from)

In [43]:
recent_vlcc_voyages_df = pd.DataFrame([v.__dict__ for v in recent_vlcc_voyages])
recent_vlcc_voyages_df['trade_id'] = recent_vlcc_voyages_df['trade_id'].fillna(-1).astype(int) 
recent_vlcc_voyages_df.tail(5)

,imo,voyage_number,vessel_type_id,vessel_class_id,vessel_status_id,commercial_operator_id,deleted,events,id,horizon_id,...,is_implied_by_ais,has_manual_entries,ballast_distance,predicted_ballast_distance,laden_distance,predicted_laden_distance,suez_crossing,panama_crossing,canakkale_crossing,bosporus_crossing
3021,9935648,2,1,85,1,1713.0,False,"(VoyageEvent(id='I979B2055SEDABEEE00', port_id...",I979B2055VEDABEEE00,1,...,None,None,233.43,NaN,5986.25,NaN,None,None,None,None
3022,9935648,3,1,85,1,946.0,False,"(VoyageEvent(id='I979B2055SEDB0E0800', port_id...",I979B2055VEDB0E0800,1,...,None,None,2755.37,NaN,366.67,NaN,None,None,None,None
3023,9935648,4,1,85,1,946.0,False,"(VoyageEvent(id='I979B2055SEDB359500', port_id...",I979B2055VEDB359500,2,...,None,None,61.76,NaN,3683.86,1790.05,None,None,None,None
3024,9935650,2,1,85,1,1450.0,False,"(VoyageEvent(id='I979B2255SEDAE67B00', port_id...",I979B2255VEDAE67B00,1,...,None,None,4739.70,NaN,4415.19,NaN,None,None,None,None
3025,9935650,3,1,85,1,1450.0,False,"(VoyageEvent(id='I979B2255SEDB359500', port_id...",I979B2255VEDB359500,2,...,None,None,4755.58,NaN,NaN,NaN,None,None,Ballast,None


In [44]:
recent_vlcc_voyages_df[~pd.isnull(recent_vlcc_voyages_df['trade_id'])]['trade_id'].head(3)

0    1
1    1
2    1
Name: trade_id, dtype: int32

In [33]:
recent_vlcc_voyages_df[pd.isnull(recent_vlcc_voyages_df['trade_id'])]

,imo,voyage_number,vessel_type_id,vessel_class_id,vessel_status_id,commercial_operator_id,deleted,events,id,horizon_id,...,is_implied_by_ais,has_manual_entries,ballast_distance,predicted_ballast_distance,laden_distance,predicted_laden_distance,suez_crossing,panama_crossing,canakkale_crossing,bosporus_crossing
3001,9924003,2,1,85,1,1713.0,False,"(VoyageEvent(id='I976DA355SEDABEEE00', port_id...",I976DA355VEDABEEE00,1,...,None,None,6660.52,NaN,3786.82,NaN,None,None,None,None
3002,9924003,3,1,85,1,1713.0,False,"(VoyageEvent(id='I976DA355SEDB0E0800', port_id...",I976DA355VEDB0E0800,2,...,None,None,2892.41,NaN,7857.80,NaN,Laden,None,None,None


In [38]:
v1 = next(x for x in recent_vlcc_voyages if x.trade_id == None)
v2 = next(x for x in recent_vlcc_voyages if x.trade_id is not None)
print(v1.trade_id, v2.trade_id)
print(type(v1.trade_id), type(v2.trade_id))

None 1
<class 'NoneType'> <class 'int'>


### Get voyages for vessel in flat format

In [13]:
# get the imo by using the vessel name
vessel = api.get_imos(VesselFilter('Seaking'))[0]
vessel

Vessel(imo=9292187, vessel_name='Seaking')

In [14]:
vessel_voyages_flat = api.get_voyages_flat(imo=vessel.imo)
vessel_voyages_flat

VoyagesFlat(voyages=(Voyage(imo=9292187, voyage_number=1, vessel_type_id=1, vessel_class_id=84, vessel_status_id=1, commercial_operator_id=1684, deleted=False, events=None, id='I8DC99BVEC1198000', horizon_id=1, horizon='Historic', latest_received_ais=None, vessel_name='Seaking', pit_vessel_name=None, vessel_type='Tanker', vessel_class='VLCC', trade='Crude', trade_id=1, vessel_status='Voyage', deadweight=318669, year_built=2005, commercial_operator='Thenamaris', start_date=datetime.datetime(2009, 2, 10, 17, 21, 24, tzinfo=datetime.timezone.utc), first_load_arrival_date=datetime.datetime(2009, 5, 8, 15, 5, 24, 91000, tzinfo=datetime.timezone.utc), end_date=datetime.datetime(2009, 10, 8, 3, 13, tzinfo=datetime.timezone.utc), charterer_id=None, charterer=None, rate=None, rate_type=None, ballast_bonus=None, ballast_bonus_type=None, cargo_type_id=19, cargo_type='Crude Oil', cargo_group_id=130000, cargo_group='Dirty', cargo_type_source='Estimated', quantity=185000.0, quantity_source='Estimate

In [15]:
vessel_voyages_df = pd.DataFrame(v.__dict__ for v in vessel_voyages_flat.voyages)
vessel_voyages_df.tail(5)

,imo,voyage_number,vessel_type_id,vessel_class_id,vessel_status_id,commercial_operator_id,deleted,events,id,horizon_id,...,is_implied_by_ais,has_manual_entries,ballast_distance,predicted_ballast_distance,laden_distance,predicted_laden_distance,suez_crossing,panama_crossing,canakkale_crossing,bosporus_crossing
61,9292187,62,1,84,1,1684,False,None,I8DC99BVEDA6FD400,1,...,None,None,4523.36,None,4586.67,None,None,None,None,None
62,9292187,63,1,84,1,1684,False,None,I8DC99BVEDA976100,1,...,None,None,4531.50,None,3968.99,None,None,None,None,None
63,9292187,64,1,84,1,1684,False,None,I8DC99BVEDABEEE00,1,...,None,None,3831.88,None,4588.66,None,None,None,None,None
64,9292187,65,1,84,1,1684,False,None,I8DC99BVEDB0E0800,1,...,None,None,4612.85,None,5273.31,None,None,None,None,None
65,9292187,66,1,84,1,1684,False,None,I8DC99BVEDB359500,2,...,None,None,2321.16,None,NaN,None,None,None,None,None


In [16]:
vessel_events_df = pd.DataFrame(v.__dict__ for v in vessel_voyages_flat.events)
vessel_events_df.tail(5)

,id,port_id,voyage_id,event_type,event_horizon,purpose,event_date,arrival_date,sailing_date,latitude,...,area_idlevel0,area_name_level0,area_idlevel1,area_name_level1,area_idlevel2,area_name_level2,area_idlevel3,area_name_level3,low_ais_density,event_details
382,I8DC99BTEDB1B3700,3711,I8DC99BVEDB0E0800,Stop,Historical,Stop,NaT,2022-12-08 15:57:28+00:00,2022-12-09 11:52:36+00:00,23.8024,...,None,None,None,None,None,None,None,None,None,None
383,I8DC99BTEDB21CE80,3778,I8DC99BVEDB0E0800,PortCall,Historical,Load,NaT,2022-12-11 23:46:36+00:00,2022-12-13 11:50:08+00:00,26.9431,...,None,None,None,None,None,None,None,None,None,None
384,I8DC99BTEDB359500,3295,I8DC99BVEDB0E0800,PortCall,Historical,Discharge,NaT,2023-01-01 03:57:12+00:00,2023-01-03 07:48:06+00:00,22.6770,...,None,None,None,None,None,None,None,None,None,None
385,I8DC99BSEDB359500,3295,I8DC99BVEDB359500,VoyageStart,Historical,Start,2023-01-03 07:48:06+00:00,NaT,NaT,22.6770,...,None,None,None,None,None,None,None,None,None,None
386,I8DC99BTEDB4FF300,3153,I8DC99BVEDB359500,Stop,Future,Stop,NaT,2023-01-20 23:00:00+00:00,2023-01-20 23:00:00+00:00,25.1975,...,None,None,None,None,None,None,None,None,None,None


In [17]:
vessel_event_details_df = pd.DataFrame(v.__dict__ for v in vessel_voyages_flat.event_details)
vessel_event_details_df.tail(5)

,id,event_id,event_detail_type,arrival_date,sailing_date,start_time_of_operation,end_time_of_operation,sts_id,geo_asset_id,geo_asset_name,latitude,longitude,other_vessel_imo,other_vessel_name
349,I8DC99BLEDB00D900,I8DC99BTEDB00D900,Jetty,2022-11-20 03:48:51+00:00,2022-11-23 11:51:53+00:00,2022-11-21 03:58:04+00:00,2022-11-23 11:51:53+00:00,None,7243,Rayong Ldpe Plant,12.6359,101.3018,NaN,None
350,I8DC99BLEDB0E0800,I8DC99BTEDB0E0800,Stop,2022-11-26 03:59:53+00:00,2022-11-27 03:57:36+00:00,NaT,NaT,None,4446,Changi Lightering Zone,1.2670,103.9481,NaN,None
351,I8DC99BLEDB1B3700,I8DC99BTEDB1B3700,Stop,2022-12-08 15:57:28+00:00,2022-12-09 11:52:36+00:00,NaT,NaT,None,5101,Mina Al Fahal Refinery,23.8024,59.4541,NaN,None
352,I8DC99BLEDB21CE80,I8DC99BTEDB21CE80,Jetty,2022-12-11 23:46:36+00:00,2022-12-13 11:50:08+00:00,2022-12-11 23:46:36+00:00,2022-12-13 11:50:08+00:00,None,5868,Saudi Aramco Ju'aymah Crude Oil Terminal,26.9431,50.0748,NaN,None
353,I8DC99BLEDB359500,I8DC99BTEDB359500,Jetty,2023-01-01 03:57:12+00:00,2023-01-03 07:48:06+00:00,2023-01-01 07:53:49+00:00,2023-01-03 07:48:06+00:00,None,5191,Huizhou Refinery,22.6770,114.6516,NaN,None


In [18]:
vessel_voyages_geos_df = pd.DataFrame(v.__dict__ for v in vessel_voyages_flat.geos)
vessel_voyages_geos_df.tail(5)

,id,name,port_id,port_name,country_id,country,area_idlevel0,area_name_level0,area_idlevel1,area_name_level1,area_idlevel2,area_name_level2,area_idlevel3,area_name_level3
233,6163,Vopak Terminal Europoort,3689,Rotterdam,173,Netherlands,24758,Continent,25016,UK Continent,25025,Mediterranean / UK Continent,25028,West
234,5744,Euro Tank Terminal B.v.,3689,Rotterdam,173,Netherlands,24758,Continent,25016,UK Continent,25025,Mediterranean / UK Continent,25028,West
235,6361,Aruba Lightering Zone,3214,San Nicolaas,21,Aruba,24746,Caribs,9,Caribs,25019,Atlantic America,25028,West
236,5745,Maatschap Europoort Terminal,3689,Rotterdam,173,Netherlands,24758,Continent,25016,UK Continent,25025,Mediterranean / UK Continent,25028,West
237,4177,Assaluyeh Anchorage,3538,Bandar Assaluyeh,115,"Iran, Islamic Republic of",24777,Arabian Gulf,2,Arabian Gulf,89,Arabian Gulf,84,East


### Get voyages for vessel class in the flat format

In [19]:
#get vessel class id for vlcc
vc = api.get_vessel_classes(VesselClassFilter('vlcc'))[0]
vlcc_id = vc.vessel_class_id

date_from = date.today() - timedelta(days=30)
recent_vlcc_voyages_flat = api.get_voyages_flat(vessel_class_id=vlcc_id, date_from=date_from)
recent_vlcc_voyages_flat

VoyagesFlat(voyages=(Voyage(imo=9174397, voyage_number=48, vessel_type_id=1, vessel_class_id=84, vessel_status_id=1, commercial_operator_id=None, deleted=False, events=None, id='I8BFD7DVEDB0E0800', horizon_id=2, horizon='Current', latest_received_ais=datetime.datetime(2023, 1, 10, 20, 59, 49, tzinfo=datetime.timezone.utc), vessel_name='Melissa Amy', pit_vessel_name=None, vessel_type='Tanker', vessel_class='VLCC', trade='Crude', trade_id=1, vessel_status='Voyage', deadweight=311168, year_built=1999, commercial_operator=None, start_date=datetime.datetime(2022, 12, 14, 3, 56, 21, tzinfo=datetime.timezone.utc), first_load_arrival_date=None, end_date=datetime.datetime(9999, 12, 31, 23, 59, 59, tzinfo=datetime.timezone.utc), charterer_id=None, charterer=None, rate=None, rate_type=None, ballast_bonus=None, ballast_bonus_type=None, cargo_type_id=None, cargo_type=None, cargo_group_id=None, cargo_group=None, cargo_type_source=None, quantity=None, quantity_source=None, laycan_from=None, laycan_to

In [20]:
print('Voyages:', len(recent_vlcc_voyages_flat.voyages))
print('Events:', len(recent_vlcc_voyages_flat.events))
print('EventDetails:', len(recent_vlcc_voyages_flat.event_details))
print('Geos:', len(recent_vlcc_voyages_flat.geos))

Voyages: 408
Events: 1456
EventDetails: 476
Geos: 254


### Get voyages for vessel incrementally 

Initial requests retrieves voyages mathcing the query criteria and a query token that can be used in the subsequent request.

In [21]:
# get the imo by using the vessel name
vessel = api.get_imos(VesselFilter('XIN YONG YANG'))[0]
vessel

Vessel(imo=9416642, vessel_name='Xin Yong Yang')

In [22]:
vessel_voyages, next_request_token = api.get_incremental_voyages(imo=vessel.imo)
len(vessel_voyages)

77

Provided the token retrieved for the previous request, the subsequent request retrieves only voyages that have been updated since the previous request. 

In [23]:
incremental_voyages, next_request_token = api.get_incremental_voyages(imo=9416642, incremental_token=next_request_token)
len(incremental_voyages)

0

The retrieved information is used to replace any modify voyages from the dataset. Note that returned voyages marked as deleted are only used to filter out voyages. 

In [24]:
updated_voyage_ids = set(v.id for v in incremental_voyages)
vessel_voyages = [v for v in vessel_voyages if v.id not in updated_voyage_ids]
new_vessel_voyages = [v for v in incremental_voyages if not v.deleted]
vessel_voyages = sorted(vessel_voyages + new_vessel_voyages, key= lambda v: v.voyage_number)
len(vessel_voyages)

77

### Get voyages for vessel class incrementally 

Voyages for vessel class are retieved and updated in the same way incrementally.

In [25]:
#get vessel class id for vlcc
vc = api.get_vessel_classes(VesselClassFilter('vlcc'))[0]
vessel_class_id = vc.vessel_class_id
vessel_class_id

84

In [26]:
date_from = date.today() - timedelta(days=30)

In [27]:
voyages, next_request_token = api.get_incremental_voyages(vessel_class_id=vessel_class_id, date_from=date_from)
len(voyages)

408

In [28]:
incremental_voyages, next_request_token = api.get_incremental_voyages(vessel_class_id=vessel_class_id, date_from=date_from, incremental_token=next_request_token)
len(incremental_voyages)

0

In [29]:
updated_voyage_ids = set(v.id for v in incremental_voyages)
voyages = [v for v in voyages if v.id not in updated_voyage_ids and not v.deleted]
new_voyages = [v for v in incremental_voyages if not v.deleted]
voyages = sorted(voyages + new_voyages, key= lambda v: (v.imo, v.voyage_number))
len(voyages)

408

### Get voyages for vessel class incrementally in the flat format

Voyages may be retrieved and updated incrementally in the flat format. 

In [30]:
#get vessel class id for vlcc
vc = api.get_vessel_classes(VesselClassFilter('vlcc'))[0]
vessel_class_id = vc.vessel_class_id
vessel_class_id

84

In [31]:
date_from = date.today() - timedelta(days=30)

In [32]:
voyages_flat, next_request_token = api.get_incremental_voyages_flat(vessel_class_id=vessel_class_id, date_from=date_from)

In [33]:
voyages = voyages_flat.voyages
events = voyages_flat.events
event_details = voyages_flat.event_details
geos = voyages_flat.geos

In [34]:
len(voyages), len(events), len(event_details), len(geos)

(408, 1456, 476, 254)

In [35]:
incremental_voyages_flat, next_request_token = api.get_incremental_voyages_flat(vessel_class_id=vessel_class_id, date_from=date_from, incremental_token=next_request_token)

In [36]:
len(incremental_voyages_flat.voyages)

1

In this case the update step is applied to voyages, events, event details and geos datasets.

In [37]:
updated_voyage_ids = set(v.id for v in incremental_voyages_flat.voyages)
voyages = [v for v in voyages if v.id not in updated_voyage_ids and not v.deleted]
new_voyages = [v for v in incremental_voyages_flat.voyages if not v.deleted]
voyages = sorted(voyages + new_voyages, key= lambda v: (v.imo, v.voyage_number))
len(voyages)

408

In [38]:
updated_event_ids = set(e.id for e in events if e.voyage_id in updated_voyage_ids)
events = [e for e in events if e.id not in updated_event_ids]
events = sorted(events + list(incremental_voyages_flat.events), key= lambda e: e.id)
len(events)

1456

In [39]:
event_details = [e for e in event_details if e.id not in updated_event_ids]
event_details = sorted(event_details + list(incremental_voyages_flat.event_details), key= lambda e: e.id)
len(event_details)

476

In [40]:
updated_geo_ids = set(g.id for g in incremental_voyages_flat.geos)
geos = [g for g in geos if g.id not in updated_geo_ids] + list(incremental_voyages_flat.geos)
len(geos)

254